In [1]:
using CSV, DataFrames

In [2]:
using FreqTables
using Statistics
using Pipe
using Dates

In [3]:
# using Pkg; Pkg.add("PyPlot")

## Read Data

In [4]:
df = CSV.read("police.csv", DataFrame);

## Understand Data

In [5]:
first(df,5)

Row,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
,Date,Time,Missing,String1?,Float64?,Float64?,String15?,String?,String31?,Bool,String?,String31?,Bool?,String15?,Bool
1,2005-01-02,01:55:00,missing,M,1985.0,20.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
2,2005-01-18,08:15:00,missing,M,1965.0,40.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
3,2005-01-23,23:15:00,missing,M,1972.0,33.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
4,2005-02-20,17:15:00,missing,M,1986.0,19.0,White,Call for Service,Other,false,missing,Arrest Driver,true,16-30 Min,false
5,2005-03-14,10:00:00,missing,F,1984.0,21.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false


In [6]:
size(df)

(91741, 15)

In [7]:
names(df)

15-element Vector{String}:
 "stop_date"
 "stop_time"
 "county_name"
 "driver_gender"
 "driver_age_raw"
 "driver_age"
 "driver_race"
 "violation_raw"
 "violation"
 "search_conducted"
 "search_type"
 "stop_outcome"
 "is_arrested"
 "stop_duration"
 "drugs_related_stop"

In [8]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,Type
1,stop_date,,2005-01-02,2010-11-04,2015-12-31,0,Date
2,stop_time,,00:00:00,11:40:00,23:59:00,0,Time
3,county_name,,,,,91741,Missing
4,driver_gender,,F,,M,5335,"Union{Missing, String1}"
5,driver_age_raw,1970.49,0.0,1980.0,8801.0,5327,"Union{Missing, Float64}"
6,driver_age,34.0113,15.0,31.0,99.0,5621,"Union{Missing, Float64}"
7,driver_race,,Asian,,White,5333,"Union{Missing, String15}"
8,violation_raw,,APB,,Warrant,5333,"Union{Missing, String}"
9,violation,,Equipment,,Speeding,5333,"Union{Missing, String31}"


In [9]:
describe(df, :mean, :median)

Row,variable,mean,median
,Symbol,Union…,Any
1,stop_date,,2010-11-04
2,stop_time,,11:40:00
3,county_name,,
4,driver_gender,,
5,driver_age_raw,1970.49,1980.0
6,driver_age,34.0113,31.0
7,driver_race,,
8,violation_raw,,
9,violation,,


In [10]:
dft = select(df, Not(["county_name","stop_duration","drugs_related_stop","is_arrested","stop_outcome","search_type"]))
first(dft, 5)

Row,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted
,Date,Time,String1?,Float64?,Float64?,String15?,String?,String31?,Bool
1,2005-01-02,01:55:00,M,1985.0,20.0,White,Speeding,Speeding,false
2,2005-01-18,08:15:00,M,1965.0,40.0,White,Speeding,Speeding,false
3,2005-01-23,23:15:00,M,1972.0,33.0,White,Speeding,Speeding,false
4,2005-02-20,17:15:00,M,1986.0,19.0,White,Call for Service,Other,false
5,2005-03-14,10:00:00,F,1984.0,21.0,White,Speeding,Speeding,false


## freqtable

In [11]:
freqtable(df.driver_race, df.driver_gender)

6×3 Named Matrix{Int64}
Dim1 ╲ Dim2 │       F        M  missing
────────────┼──────────────────────────
Asian       │     513     1746        0
Black       │    2580     9664        0
Hispanic    │    1871     7636        0
Other       │      26      214        0
White       │   18521    43635        2
missing     │       0        0     5333

In [12]:
freqtable(df.driver_race, df.driver_gender, skipmissing=true)

5×2 Named Matrix{Int64}
Dim1 ╲ Dim2 │     F      M
────────────┼─────────────
Asian       │   513   1746
Black       │  2580   9664
Hispanic    │  1871   7636
Other       │    26    214
White       │ 18521  43635

### prop (Proportion)

In [13]:
prop(freqtable(df.driver_race, df.driver_gender))

6×3 Named Matrix{Float64}
Dim1 ╲ Dim2 │           F            M      missing
────────────┼──────────────────────────────────────
Asian       │  0.00559183    0.0190318          0.0
Black       │   0.0281226      0.10534          0.0
Hispanic    │   0.0203944    0.0832343          0.0
Other       │ 0.000283407   0.00233265          0.0
White       │    0.201884     0.475632   2.18005e-5
missing     │         0.0          0.0     0.058131

In [14]:
prop(freqtable(df.driver_race, df.driver_gender), margins=1)

6×3 Named Matrix{Float64}
Dim1 ╲ Dim2 │          F           M     missing
────────────┼───────────────────────────────────
Asian       │   0.227092    0.772908         0.0
Black       │   0.210715    0.789285         0.0
Hispanic    │   0.196802    0.803198         0.0
Other       │   0.108333    0.891667         0.0
White       │   0.297966    0.702001  3.21761e-5
missing     │        0.0         0.0         1.0

In [15]:
prop(freqtable(df.driver_race, df.driver_gender), margins=2)

6×3 Named Matrix{Float64}
Dim1 ╲ Dim2 │           F            M      missing
────────────┼──────────────────────────────────────
Asian       │   0.0218196    0.0277606          0.0
Black       │    0.109736     0.153653          0.0
Hispanic    │   0.0795798     0.121409          0.0
Other       │  0.00110587    0.0034025          0.0
White       │    0.787759     0.693775  0.000374883
missing     │         0.0          0.0     0.999625

## isequal

In [16]:
dft = df[isequal.(df.driver_race, "Asian"), :];
first(dft,5)

Row,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
,Date,Time,Missing,String1?,Float64?,Float64?,String15?,String?,String31?,Bool,String?,String31?,Bool?,String15?,Bool
1,2005-07-13,16:20:00,missing,M,1979.0,26.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
2,2005-07-28,12:37:00,missing,F,1979.0,26.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
3,2005-09-30,14:00:00,missing,M,1962.0,43.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
4,2005-10-01,23:25:00,missing,M,1974.0,31.0,Asian,Speeding,Speeding,false,missing,Citation,false,16-30 Min,false
5,2005-10-03,02:20:00,missing,F,1950.0,55.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false


In [17]:
df[isequal.(df.driver_race, "Asian"), "stop_outcome"] |> freqtable

6-element Named Vector{Int64}
Dim1             │ 
─────────────────┼─────
Arrest Driver    │   36
Arrest Passenger │    5
Citation         │ 2108
N/D              │   11
No Action        │   15
Warning          │   84

In [18]:
df[isequal.(df.driver_race, "Asian"), "stop_outcome"] |> freqtable |> prop

6-element Named Vector{Float64}
Dim1             │ 
─────────────────┼───────────
Arrest Driver    │  0.0159363
Arrest Passenger │ 0.00221337
Citation         │   0.933156
N/D              │ 0.00486941
No Action        │ 0.00664011
Warning          │  0.0371846

## groupby

In [19]:
groupby(df, "driver_race")

Row,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
,Date,Time,Missing,String1?,Float64?,Float64?,String15?,String?,String31?,Bool,String?,String31?,Bool?,String15?,Bool
1,2005-01-02,01:55:00,missing,M,1985.0,20.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
2,2005-01-18,08:15:00,missing,M,1965.0,40.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
3,2005-01-23,23:15:00,missing,M,1972.0,33.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
4,2005-02-20,17:15:00,missing,M,1986.0,19.0,White,Call for Service,Other,false,missing,Arrest Driver,true,16-30 Min,false
5,2005-03-14,10:00:00,missing,F,1984.0,21.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
6,2005-04-01,17:30:00,missing,M,1969.0,36.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
7,2005-06-06,13:20:00,missing,F,1986.0,19.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
8,2005-07-13,15:45:00,missing,M,1970.0,35.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
9,2005-07-13,19:00:00,missing,F,1966.0,39.0,White,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false


In [20]:
dft = groupby(df, "driver_race")[("Asian",)]
first(dft, 5)

Row,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
,Date,Time,Missing,String1?,Float64?,Float64?,String15?,String?,String31?,Bool,String?,String31?,Bool?,String15?,Bool
1,2005-07-13,16:20:00,missing,M,1979.0,26.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
2,2005-07-28,12:37:00,missing,F,1979.0,26.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
3,2005-09-30,14:00:00,missing,M,1962.0,43.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false
4,2005-10-01,23:25:00,missing,M,1974.0,31.0,Asian,Speeding,Speeding,false,missing,Citation,false,16-30 Min,false
5,2005-10-03,02:20:00,missing,F,1950.0,55.0,Asian,Speeding,Speeding,false,missing,Citation,false,0-15 Min,false


In [21]:
df_grp = groupby(df, "driver_race", skipmissing=true)
combine(df_grp, "search_conducted" => mean, nrow)

Row,driver_race,search_conducted_mean,nrow
,String15?,Float64,Int64
1,White,0.0284436,62158
2,Black,0.0645214,12244
3,Asian,0.0225764,2259
4,Hispanic,0.0614284,9507
5,Other,0.0125,240


In [22]:
@pipe groupby(df, "driver_race", skipmissing=true) |>
      combine(_, "search_conducted" => mean, nrow)

Row,driver_race,search_conducted_mean,nrow
,String15?,Float64,Int64
1,White,0.0284436,62158
2,Black,0.0645214,12244
3,Asian,0.0225764,2259
4,Hispanic,0.0614284,9507
5,Other,0.0125,240


## Column modifications

In [23]:
first(df.stop_date, 2)

2-element Vector{Date}:
 2005-01-02
 2005-01-18

In [24]:
last(df.stop_date, 2)

2-element Vector{Date}:
 2015-12-31
 2015-12-31

In [25]:
dft = select(df, :stop_date => ByRow(year))
first(dft,2), last(dft, 2)

(2×1 DataFrame
 Row │ stop_date_year 
     │ Int64          
─────┼────────────────
   1 │           2005
   2 │           2005, 2×1 DataFrame
 Row │ stop_date_year 
     │ Int64          
─────┼────────────────
   1 │           2015
   2 │           2015)

Rename column

In [26]:
dft = select(df, :stop_date => ByRow(year) => :year)
first(dft,2)

Row,year
,Int64
1,2005
2,2005


Complex transformation using everything seen till now

In [27]:
dft = @pipe select(df, :stop_date => ByRow(year) => :year, :search_conducted ) |>
            groupby(_, :year) |>
            combine(_, "search_conducted" => sum, nrow => :count)

Row,year,search_conducted_sum,count
,Int64,Int64,Int64
1,2005,128,2558
2,2006,407,10639
3,2007,364,9476
4,2008,294,8752
5,2009,381,7908
6,2010,290,7561
7,2011,276,8126
8,2012,354,10970
9,2013,205,7924


### transform

In [28]:
dft = transform(df, :stop_time => ByRow(hour) => :hour)

first(dft[!, [:stop_time, :hour]], 5)

Row,stop_time,hour
,Time,Int64
1,01:55:00,1
2,08:15:00,8
3,23:15:00,23
4,17:15:00,17
5,10:00:00,10


In [29]:
dft = transform(df, :stop_time => ByRow(hour) => :hour)

first(dft[!, [:stop_time, :hour]], 5)

Row,stop_time,hour
,Time,Int64
1,01:55:00,1
2,08:15:00,8
3,23:15:00,23
4,17:15:00,17
5,10:00:00,10
